In [1]:
from sklearn.decomposition import PCA
from sklearn.mixture import GMM
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import pickle
import gc
import math
import DomainManipulation, DataLoader

C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\numba\errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
res = DataLoader.readFromDirectory(r"data/train_audio/aldfly", filecount=2)

reading file: data/train_audio/aldfly/XC135456.mp3
reading file: data/train_audio/aldfly/XC135457.mp3


In [ ]:
def trainGMM(samples, n_components = 5):
    model = GMM(n_components=n_components, covariance_type='full')
    model.fit(samples)
    return model

def multiTo2DVis(data, plot=True):
    pca = PCA(n_components=3)
    pca.fit(data)

    print("PCA explained variane: {}".format(pca.explained_variance_ratio_))

    data_reduced = pca.transform(data)

    ax, ay = data_reduced[:,0].flatten(), data_reduced[:,1].flatten()
    import matplotlib.pyplot as plt
    
    
    return plt.scatter(ax, ay, alpha=0.003, c = [1,1,0,1]) if plot else ax,ay 

In [ ]:
mfcc_directory = 'data/mfcc_50/'
picklefiles = os.listdir(mfcc_directory)

In [ ]:
birdPriorModel = {}

for i,picklefile in enumerate(picklefiles[:10]):
    print("learning {}: {}".format(i,picklefile))
    
    mfcc_samples_path = os.path.join(mfcc_directory,picklefile)
    mfcc_samples = pickle.load(open(mfcc_samples_path, 'rb')).T
    pca = PCA(n_components=3)
    mfcc_samples = pca.fit_transform(mfcc_samples)
    
    norm2 = np.linalg.norm(mfcc_samples, axis=1)
    mfcc_samples = mfcc_samples / norm2[:, np.newaxis]
    
    np.random.shuffle(mfcc_samples)
    model = trainGMM(mfcc_samples)
    birdPriorModel[picklefile.split(".")[0]] = model
    
    del(mfcc_samples)
    gc.collect()

pickle.dump(birdPriorModel,open("data/birdPriorModel.pickle",'wb'))

In [ ]:
# gather all means
allMeans = None
allVars = None
birdValues = []
check = True
for bird,gmmModel in birdPriorModel.items():
    allMeans = gmmModel.means_ if check else np.concatenate([allMeans,gmmModel.means_])
    
    var = gmmModel.covars_
#     vardets = np.mean(var,axis=1)
    vardets = np.array( [np.linalg.det(var[i,:,:]) for i in range(var.shape[0])] )
    allVars = vardets if check else np.concatenate([allVars,vardets])
    
    birdValues = birdValues + [bird]*var.shape[0]
    
    check = False

# assign colors to each bird
# colors = cm.rainbow(np.linspace(0, 1, len(ys)))
birdSet = list(set(birdValues))
birdCodes = [birdSet.index(eachBird)/len(birdSet) for eachBird in birdValues]
colors = cm.rainbow(np.array(birdCodes))


# visualize bird gmm means
ax, ay = multiTo2DVis(allMeans, plot=False)
plt.scatter(ax, ay, alpha = 1, c = colors)
plt.show()

In [ ]:
mfcc_samples_path = os.path.join(mfcc_directory, "amepip.pickle")
mfcc_samples = pickle.load(open(mfcc_samples_path, 'rb')).T
pca = PCA(n_components=3)
mfcc_samples = pca.fit_transform(mfcc_samples)

In [ ]:
# score priors
def predictPrior(priorModel, samples):
    birdScores = None
    check = True
    for bird,gmmModel in priorModel.items():
        birdScore = gmmModel.score(samples)
        birdScore = birdScore.reshape(len(birdScore),1)
        birdScores = birdScore if check else np.concatenate([birdScores,birdScore], axis=1)
        check=False
        del(birdScore)
    return birdScores
gc.collect()

logPriors = predictPrior(birdPriorModel, mfcc_samples)

vfunc = np.vectorize(lambda x: math.exp(x))
logPriorRowMeans = np.max(logPriors, axis=1)
logPriors = logPriors - logPriorRowMeans[:, np.newaxis]
logPriors = vfunc(logPriors)
[print(str(i) +":"+ str(x)) for i,x in enumerate(np.mean(logPriors, axis=0))]

sum_of_rows = logPriors.sum(axis=1)
priorProbs = logPriors / sum_of_rows[:, np.newaxis]
print(np.mean(priorProbs, axis=0))
priorProbs.shape

In [ ]:
import DataLoader
y_all = DataLoader.readFromDirectory('data/train_audio/amepip')

y = np.array([])

for i in range(y_all.shape[0]):
    try:
        y = np.concatenate([y, y_all[i]])
    except:
        y = y_all[i,:]

        
amp = DomainManipulation.rollingMean([math.fabs(yy) for yy in y], 1000, samples = mfcc_samples.shape[0])

In [ ]:
# plot segmentation
plt.clf()

start = 100000
window = 2000

plt.subplot(211)
plt.plot(range(len(amp[start:start+window])), amp[start:start+window], 'b')

logPriors = predictPrior(birdPriorModel, mfcc_samples)
labels = priorProbs[:,6].flatten()
# labels = logPriors[:,6].flatten()
plt.subplot(212)
plt.plot(range(len(labels[start:start+window])), labels[start:start+window], 'r')

plt.show()

In [ ]:
np.mean(logPriors, axis=0).shape
logPriors.shape